For individual tensors, we can directly invoke the load and save functions to read and write them respectively. Both functions require that we supply a name, and save requires as input the variable to be saved.

In [2]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

In [3]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

We can store a list of tensors and read them back into memory.

In [4]:
y = torch.zeros(4)
torch.save([x,y], 'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

We can even write and read a dictionary that maps from strings to tensors. This is convenient when we want to read or write all the weights in a model.

In [5]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

Saving individual weight vectors (or other tensors) is useful, but it gets very tedious if we want to save (and later load) an entire model. After all, we might have hundreds of parameter groups sprinkled throughout. For this reason the DL framework provides built-in functionalities to load and save entire networks. An important detail to note is that htis saves model parameters and not the entire model. For example, if we have a 3-layer MLP, we need to specify the architecture separaelty. The reason for this is that the models themselves can contain arbitary code, hence they cannot be serialized as naturally. Thus, in order to reinstate a model, we need to generate the architecture in code and then load the parameters from sidk.

In [7]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.LazyLinear(256)
        self.output = nn.LazyLinear(10)
        
    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))
    
net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [8]:
# Next, we store the parameters of the model as a file with the name "mlp.params"
torch.save(net.state_dict(), 'mlp.params')

In [9]:
# To recover the model, we instantiate a clone of the original MLP.
# Instead of randomly initializing the model parameters,
# we read the parameters stored in the file directly.
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

/opt/miniconda3/envs/d2l/lib/python3.11/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


MLP(
  (hidden): LazyLinear(in_features=0, out_features=256, bias=True)
  (output): LazyLinear(in_features=0, out_features=10, bias=True)
)

In [10]:
# Since both instances have the same model parameters, the computational result of the same input X should be the same.
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])